In [2]:
import nltk

import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tag import pos_tag
from collections import defaultdict
from math import log

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

documents = [
    "Natural Language Processing enables computers to understand human language.",
    "Tokenization and stemming are crucial steps in text preprocessing.",
    "TF-IDF is a statistical measure used to evaluate how important a word is to a docum"
]

def preprocess_document(doc):
    # Tokenization
    tokens = word_tokenize(doc.lower())

    # POS Tagging
    pos_tags = pos_tag(tokens)

    # Stopword removal
    stop_words = set(stopwords.words('english'))
    filtered = [word for word, tag in pos_tags if word.isalpha() and word not in stop_words]

    # Stemming
    stemmer = PorterStemmer()
    stemmed = [stemmer.stem(word) for word in filtered]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    pos_map = {'N': 'n', 'V': 'v', 'R': 'r', 'J': 'a'}
    lemmatized = []
    for word, tag in pos_tags:
        if word.isalpha() and word not in stop_words:
            pos = pos_map.get(tag[0], 'n')
            lemmatized.append(lemmatizer.lemmatize(word, pos=pos))

    return {
        'original': doc,
        'tokens': tokens,
        'pos_tags': pos_tags,
        'filtered': filtered,
        'stemmed': stemmed,
        'lemmatized': lemmatized
    }

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def build_tfidf(docs):
    vocabulary = set()
    doc_tokens = []

    for doc in docs:
        tokens = doc['lemmatized']
        doc_tokens.append(tokens)
        vocabulary.update(tokens)

    vocabulary = sorted(vocabulary)
    num_docs = len(docs)

    # Term Frequency (TF)
    tf = []
    for tokens in doc_tokens:
        doc_tf = defaultdict(int)
        for word in tokens:
            doc_tf[word] += 1
        tf.append(doc_tf)

    # Document Frequency (DF)
    df = defaultdict(int)
    for word in vocabulary:
        for tokens in doc_tokens:
            if word in tokens:
                df[word] += 1

    # Inverse Document Frequency (IDF)
    idf = {}
    for word in vocabulary:
        idf[word] = log(num_docs / (1 + df[word]))

    # TF-IDF Calculation
    tfidf_vectors = []
    for doc_tf in tf:
        tfidf = []
        for word in vocabulary:
            tf_value = doc_tf.get(word, 0)
            tfidf_value = tf_value * idf[word]
            tfidf.append(tfidf_value)
        tfidf_vectors.append(tfidf)

    return {
        'vocabulary': vocabulary,
        'tfidf': tfidf_vectors
    }

# Processing all documents
preprocessed = [preprocess_document(doc) for doc in documents]
tfidf_result = build_tfidf(preprocessed)

# Output TF-IDF Matrix
print("\nTF-IDF Matrix:")
header = "Term".ljust(15) + "\t" + "\t".join([f"Doc {i+1}" for i in range(len(documents))])
print(header)

for i, term in enumerate(tfidf_result['vocabulary']):
    row = [f"{tfidf_result['tfidf'][doc_idx][i]:.4f}" for doc_idx in range(len(documents))]
    print(term.ljust(15) + "\t" + "\t".join(row))

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\HP/nltk_data'
    - 'C:\\Users\\HP\\Desktop\\sample_project_2\\env\\nltk_data'
    - 'C:\\Users\\HP\\Desktop\\sample_project_2\\env\\share\\nltk_data'
    - 'C:\\Users\\HP\\Desktop\\sample_project_2\\env\\lib\\nltk_data'
    - 'C:\\Users\\HP\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
